In [ ]:
!pip install pydantic openai

In [ ]:
!pip install pydantic==1.10.11

In [11]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00


In [1]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [2]:
from pydantic import BaseModel
from typing import List, Optional

class TenantRepresentativeDetails(BaseModel):
    FirstName: str
    LastName:  str
    Email:     str
    Phone:     Optional[str]

class CompanyDetails(BaseModel):
    CompanyName:        str
    Industry:           str
    CompanySize:        str
    GrowthStage:        str
    CurrentNeighborhood: Optional[str]

class PropertyPreferences(BaseModel):
    PropertyType:            str
    PreferredNeighborhood:   str
    EstimatedOrStatedBudget: str
    MustHaves:               List[str]
    NiceToHaves:             List[str]
    SpaceSize:               str
    PreferredLeaseTerm:      Optional[str]

class TenantProfile(BaseModel):
    TenantRepresentativeDetails: TenantRepresentativeDetails
    CompanyDetails:              CompanyDetails
    FirstInteraction:            str
    LastInteraction:             str
    DecisionMakerRole:           Optional[str]
    PropertyPreferences:         PropertyPreferences
    MovingTimeline:              str
    PainPoints:                  List[str]
    UrgencyScore:                int
    Outcome:                     str

class EmailChainRecord(BaseModel):
    email_chain:   List[str]
    tenant_profile: TenantProfile

In [4]:
import json

data = json.load(open("/content/ one_chain.json"))

#remap the tenant_profile keys themselves
tp = data["tenant_profile"]
tp["TenantRepresentativeDetails"] = tp.pop("Tenant Representative Details")
tp["CompanyDetails"]              = tp.pop("Company Details")
tp["FirstInteraction"]            = tp.pop("First Interaction")
tp["LastInteraction"]             = tp.pop("Last Interaction")
tp["DecisionMakerRole"]           = tp.pop("Decision-Maker Role")
tp["PropertyPreferences"]         = tp.pop("Property Preferences")
tp["MovingTimeline"]              = tp.pop("Moving Timeline")
tp["PainPoints"]                  = tp.pop("Pain Points")
tp["UrgencyScore"]                = tp.pop("Urgency Score")
data["tenant_profile"] = tp

#TenantRepresentativeDetails
rep = tp["TenantRepresentativeDetails"]
rep["FirstName"] = rep.pop("First Name")
rep["LastName"]  = rep.pop("Last Name")

tp["TenantRepresentativeDetails"] = rep

#CompanyDetails
cd = tp["CompanyDetails"]
cd["CompanyName"]        = cd.pop("Company Name")
cd["CompanySize"]        = cd.pop("Company Size")
cd["GrowthStage"]        = cd.pop("Growth Stage")
cd["CurrentNeighborhood"]= cd.pop("Current Neighborhood")
tp["CompanyDetails"] = cd

#PropertyPreferences
pp = tp["PropertyPreferences"]
pp["PropertyType"]            = pp.pop("Property Type")
pp["PreferredNeighborhood"]   = pp.pop("Preferred Neighborhood")
pp["EstimatedOrStatedBudget"] = pp.pop("Estimated or Stated Budget")
pp["MustHaves"]               = pp.pop("Must-Haves")
pp["NiceToHaves"]             = pp.pop("Nice-to-Haves")
pp["SpaceSize"]               = pp.pop("Space Size")
pp["PreferredLeaseTerm"]      = pp.pop("Preferred Lease Term")
tp["PropertyPreferences"] = pp


record = EmailChainRecord(**data)
print("✅ Schema validated")
record

✅ Schema validated


EmailChainRecord(email_chain=['From: Alex Carter <alex.carter@pinnaclerealty.com>\\nTo: Lauren Hayes <lauren@pixelcraft.design>\\nSubject: Unique Office Spaces in Manhattan\\n\\nHi Lauren,\\n\\nI’m Alex Carter with Pinnacle Realty. Are you considering a new space for PixelCraft? I specialize in unique properties that might fit your creative vibe.\\n\\nBest,\\nAlex Carter', 'From: Lauren Hayes <lauren@pixelcraft.design>\\nTo: Alex Carter <alex.carter@pinnaclerealty.com>\\nSubject: Re: Unique Office Spaces in Manhattan\\n\\nAlex,\\n\\nWe’re a design firm, 25 people, and we need a space under $20k/month with a freight elevator, lots of light, and a unique feel. Don’t waste my time with anything else.\\n\\n— Lauren', 'From: Alex Carter <alex.carter@pinnaclerealty.com>\\nTo: Lauren Hayes <lauren@pixelcraft.design>\\nSubject: Re: Unique Office Spaces in Manhattan\\n\\nHi Lauren,\\n\\nI’ve got a 2,000 sqft loft in East Village—$18k/month, exposed brick, skylights. No freight elevator, but the

In [5]:
record.tenant_profile

TenantProfile(TenantRepresentativeDetails=TenantRepresentativeDetails(FirstName='Lauren', LastName='Hayes', Email='lauren@pixelcraft.design', Phone=None), CompanyDetails=CompanyDetails(CompanyName='PixelCraft', Industry='Creative', CompanySize='Small (20-75 employees)', GrowthStage='Startup', CurrentNeighborhood=None), FirstInteraction='31-10-2025', LastInteraction='07-11-2025', DecisionMakerRole=None, PropertyPreferences=PropertyPreferences(PropertyType='Office', PreferredNeighborhood='East Village, Chelsea', EstimatedOrStatedBudget='Under $20,000/month', MustHaves=['Freight elevator', 'Unique space', 'Natural light'], NiceToHaves=[], SpaceSize='2,000-2,500 sqft', PreferredLeaseTerm=None), MovingTimeline='Exploratory', PainPoints=['Budget constraints', 'Broker incompetence'], UrgencyScore=3, Outcome='Tenant terminates relationship with broker in hostile exchange')

In [6]:
def run_rules(tp):
    suggestions = []

    # R1: high urgency → immediate tour
    if tp.UrgencyScore >= 8 and "Immediate" in tp.MovingTimeline:
        suggestions.append(
            "Schedule tour within next 24 hours; follow up by phone if no confirmation in 2 hours."
        )

    # R2: budget conflict → negotiate incentives
    if "Under" in tp.PropertyPreferences.EstimatedOrStatedBudget and "$" in tp.PropertyPreferences.EstimatedOrStatedBudget:
        suggestions.append(
            "Negotiate rent concessions or landlord incentives (e.g., free month) rather than expanding search."
        )

    # R3: pet‑friendly must-have → highlight certification
    if any("pet" in m.lower() for m in tp.PropertyPreferences.MustHaves):
        suggestions.append(
            "Filter listings for pet‑friendly certification; highlight this feature in outreach email."
        )

    # R4: decision‑maker is COO or Founder → include ROI/floorplans
    role = (tp.DecisionMakerRole or "").lower()
    if "coo" in role or "founder" in role:
        suggestions.append(
            "Include detailed floor plans and ROI analysis in the next message."
        )

    # R5: tenant tone hesitant or budget dispute → dual‑option offering
    if tp.UrgencyScore < 5 or any("still" in point.lower() or "considering" in point.lower()
                                  for point in tp.PainPoints):
        suggestions.append(
            "Offer two alternative listings: one slightly under budget, one with added amenity incentive."
        )

    # R6: noisy neighbors or HVAC issues → prioritize sound‑proofing
    if any("noisy" in p.lower() or "hvac" in p.lower() for p in tp.PainPoints):
        suggestions.append(
            "Prioritize buildings with sound‑proofing specs and documented HVAC maintenance."
        )

    # R7: short lease term preference → propose flexible terms
    term = tp.PropertyPreferences.PreferredLeaseTerm or ""
    if "short" in term.lower() or "1-3" in term.lower():
        suggestions.append(
            "Emphasize short‑term lease flexibility and renewal options in your proposal."
        )

    # R8: large space size requested → suggest space‑planning service
    size = tp.PropertyPreferences.SpaceSize or ""
    if any(label in size.lower() for label in ["5000", "large", "5,000"]):
        suggestions.append(
            "Offer a complimentary space‑planning consultation to optimize large‑floor layouts."
        )

    return suggestions

In [7]:
print(run_rules(record.tenant_profile))

['Negotiate rent concessions or landlord incentives (e.g., free month) rather than expanding search.', 'Offer two alternative listings: one slightly under budget, one with added amenity incentive.']


In [8]:
import openai, json, os
openai.api_key = os.environ["OPENAI_API_KEY"]

def llm_summarize(sample_dict):
    prompt = f"""
You are an email‑thread summarization engine.
Input JSON:
{json.dumps(sample_dict, indent=2)}
Respond only with valid JSON of the form:
{{ "summary": {{…}}, "suggestions": […] }}
"""
    resp = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role":"user","content":prompt}],
        temperature=0.2
    )
    return json.loads(resp.choices[0].message.content)

out = llm_summarize(data)
print(json.dumps(out, indent=2))

{
  "summary": {
    "outcome": "Tenant terminates relationship with broker in hostile exchange",
    "pain_points": [
      "Budget constraints",
      "Broker incompetence"
    ],
    "final_interaction_date": "07-11-2025"
  },
  "suggestions": []
}


In [9]:
#integrate rules
combined = out.copy()
combined["rules"] = run_rules(record.tenant_profile)
print(json.dumps(combined, indent=2))

{
  "summary": {
    "outcome": "Tenant terminates relationship with broker in hostile exchange",
    "pain_points": [
      "Budget constraints",
      "Broker incompetence"
    ],
    "final_interaction_date": "07-11-2025"
  },
  "suggestions": [],
  "rules": [
    "Negotiate rent concessions or landlord incentives (e.g., free month) rather than expanding search.",
    "Offer two alternative listings: one slightly under budget, one with added amenity incentive."
  ]
}


In [12]:
import gradio as gr

def analyze_chain(raw_json):
    try:
        sample = json.loads(raw_json)
        #remap
        tp = sample["tenant_profile"]
        mapping_tp = {
            "Tenant Representative Details":"TenantRepresentativeDetails",
            "Company Details":"CompanyDetails",
            "First Interaction":"FirstInteraction",
            "Last Interaction":"LastInteraction",
            "Decision-Maker Role":"DecisionMakerRole",
            "Property Preferences":"PropertyPreferences",
            "Moving Timeline":"MovingTimeline",
            "Pain Points":"PainPoints",
            "Urgency Score":"UrgencyScore"
        }
        for old, new in mapping_tp.items():
            tp[new] = tp.pop(old)

        #TRD
        rep = tp["TenantRepresentativeDetails"]
        rep["FirstName"] = rep.pop("First Name")
        rep["LastName"]  = rep.pop("Last Name")
        tp["TenantRepresentativeDetails"] = rep
        cd = tp["CompanyDetails"]
        mapping_cd = {
            "Company Name":"CompanyName",
            "Company Size":"CompanySize",
            "Growth Stage":"GrowthStage",
            "Current Neighborhood":"CurrentNeighborhood"
        }
        for old, new in mapping_cd.items():
            cd[new] = cd.pop(old)
        tp["CompanyDetails"] = cd
        #PP
        pp = tp["PropertyPreferences"]
        mapping_pp = {
            "Property Type":"PropertyType",
            "Preferred Neighborhood":"PreferredNeighborhood",
            "Estimated or Stated Budget":"EstimatedOrStatedBudget",
            "Must-Haves":"MustHaves",
            "Nice-to-Haves":"NiceToHaves",
            "Space Size":"SpaceSize",
            "Preferred Lease Term":"PreferredLeaseTerm"
        }
        for old, new in mapping_pp.items():
            pp[new] = pp.pop(old)
        tp["PropertyPreferences"] = pp
        sample["tenant_profile"] = tp

        record = EmailChainRecord(**sample)
        out = llm_summarize(sample)
        out["rules"] = run_rules(record.tenant_profile)

        return json.dumps(out, indent=2)

    except Exception as e:
        return f"⚠️ Error: {type(e).__name__}: {e}"

In [13]:
import matplotlib.pyplot as plt
import gradio as gr

def analyze_chain_with_plot(raw_json):
    result_json = analyze_chain(raw_json)

    # parse back into dict so we can chart
    data = json.loads(result_json)
    profile = json.loads(raw_json)["tenant_profile"]
    # pull numeric values
    urgency = profile["Urgency Score"]
    # convert MovingTimeline to a number of days
    mt = profile["Moving Timeline"]
    days = 0
    if "Immediate" in mt:
        days = 10
    elif "30" in mt:
        days = 30
    elif "Short‑term" in mt or "3-6" in mt:
        days = 90
    elif "Exploratory" in mt:
        days = 180
    #bar chart
    fig, ax = plt.subplots()
    ax.bar(["UrgencyScore","TimelineDays"], [urgency, days])
    ax.set_ylabel("Value")
    ax.set_title("Tenant Urgency vs Timeline")
    fig.tight_layout()

    return result_json, fig


iface = gr.Interface(
    fn=analyze_chain_with_plot,
    inputs=gr.Textbox(lines=15, label="Paste chain JSON here"),
    outputs=[
      gr.Code(label="Summary + Suggestions"),
      gr.Plot(label="Urgency vs Timeline")
    ],
    title="Email Chain Analyzer"
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fbbe8870a9d468371a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
